In [10]:
import tensorflow as tf 

## Design Flow
1. Assemble a graph
2. Use a session to execute the Graph

Tensor - n Dimensional array

Basic Operations - ADD  
a = tf.add(3,5)

A computation graph is created where-: 
1. Nodes - Operators,Variables and Constants
2. Edges - Tensors

Tensor -> data
TensorFlow = Tensor + Flow = Data + Flow  
Mind = Blown! :P


In [11]:
a = tf.add(3,5)
print(a)

Tensor("Add:0", shape=(), dtype=int32)


This output because 'a' is currently a node in the graph generated by TensorFlow. Value of a can only be obtained after the graph is executed!

## Steps - :
1. Assemble a graph.
2. Create a Session and assign it to a variable, sess (to call it later).
3. Within session, evaluate graphs to do the necessary computations.

In [12]:
a = tf.add(3,5)
sess = tf.Session()
print(sess.run(a))
sess.close()

8


In [13]:
a = tf.add(3,5)
with tf.Session() as sess:
    print(sess.run(a))
    

8


## tf.Session()
A Session object encapsulates the environment in which Operation objects are executed, and Tensor objects are evaluated.  

Session will aso allocate memory to store the current values of variables

In [14]:
#More Graphs
x = 2 
y = 3
op1 = tf.add(x,y)
op2 = tf.multiply(x,y)
op3 = tf.pow(op2,op1)
with tf.Session() as sess:
    op3 = sess.run(op3)
sess.close()

In [15]:
x = 2
y = 3
add_op = tf.add(x,y)
mul_op = tf.multiply(x,y)
useless = tf.multiply(x, add_op)
pow_op = tf.pow(add_op,mul_op)
with tf.Session() as sess:
    z = sess.run(pow_op)
sess.close()

In the above, value of 'useless' will not be computed as we dont require it for the computation of 'pow_op'

-> General Syntax - tf.Session(fetches, feed_dict = None, options = None, run_metadata = None)  
fetches - list of tensors whoes values you want

In [16]:
x = 2
y = 3
add_op = tf.add(x,y)
mul_op = tf.multiply(x,y)
useless = tf.multiply(x, add_op)
pow_op = tf.pow(add_op,mul_op)
with tf.Session() as sess:
    z = sess.run([pow_op, useless])
sess.close()

### It is possible to break graphs into several chunks and run them across multiple CPUs, GPUs, TPUs or other devices

Example - AlexNet


## To get available devices on which you can perform computations


In [17]:
from tensorflow.python.client import device_lib

def get_available_cpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'CPU']

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_cpus() + get_available_gpus())

['/device:CPU:0']


## Distributed Computation
To put a graph on a specific CPU or GPU


In [19]:
#Create a Graph
with tf.device('/CPU:0'):
    a = tf.constant([1.0,2.0,3.0,4.0,5.0,6.0], name ='a')
    b = tf.constant([1.0,2.0,3.0,4.0,5.0,6.0], name ='b')
    c = tf.multiply(a,b)

In [20]:
#Create a Session with og_device_placement set to True
sess = tf.Session(config = tf.ConfigProto(log_device_placement=True))

In [21]:
#Run the Operation
print(sess.run(c))

[ 1.  4.  9. 16. 25. 36.]


## Buiding more than one graph
Generally you dont need more than one graph as Session only runs the default graph.

1. Multiple graphs require mutiple session, each will try to use al available resources by default
2. Can't pass data betweeen them without passing them through python/numpy, which doesn't  work in distributed network
3. Its better to have disconnect graphs within one graph

But if you really want to make mutiple graphs then follow :


In [32]:
#Create a graph
g = tf.Graph()

In [33]:
with g.as_default():
    q = tf.add(3,5)
sess = tf.Session(graph = g)
sess.run(q)

8

In [34]:
#To handle the default graph
w = tf.get_default_graph()


In [35]:
#Note : Dont mix dfault graphs and user created graphs
g = tf.Graph()

#Add ops to the default graph
a = tf.constant(3)

#Add ops to the user created graph
with g.as_default():
    b = tf.constant(5)

In [37]:
g1 = tf.get_default_graph()
g2 = tf.Graph()

# Add operations to the default graph
with g1.as_default():
    a = tf.constant(3)

# Add operations to the user created graphs
with g2.as_default():
    b = tf.constant(5)

Managing of two graphs is highly prone to erroors as you can mess up Tensors and Operations of different Graphs.

Hence, Practice of using different graphs is avoided.

## Why Graphs?
While working with TensorFlow, a question might arise that, Why Graphs?  
1. Save Computations - Only run subgraphs which leads to values you want to fetch
2. Break coputation into small, differential pieces to faciitace auto-differentiation
3. Facilitate distributed computations
4. Many algorithms are visualized and taught as directed graphs